# Feature Selection on the 111001001 dataset

In this notebook we examine the result of dropping subsets of features that are linearly dependent or correlated to some degree.  We're specifically working on the dataset formed by dropping (31, 496, 524, 917, 1299), since that resulted in the lowest validation error.

In [6]:
import itertools
import numpy as np
import pandas as pd

pd.set_option('display.precision',20)
pd.set_option('display.max_colwidth',100)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_predict, KFold, cross_val_score, \
                                    GridSearchCV, RandomizedSearchCV, ShuffleSplit 
from time import time
from scipy.stats import randint as sp_randint

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib import pyplot
rcParams['figure.figsize'] = 12, 4
%matplotlib inline

In [2]:
# def to compare goodness of fit on training set
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [11]:
# Cross-validation sets
kfold = KFold(n_splits=10, random_state=7)

# We are using LassoLarsCV as part of our metric
lr = linear_model.LassoLarsCV(verbose=False, max_iter=5000,precompute='auto', cv=kfold, max_n_alphas=1000, n_jobs=-1)

In [4]:
df = pd.read_csv("./input/train_tidy_111001001.csv")

In [7]:
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=573)

In [8]:
X = df.values

In [9]:
for train_idx, validation_idx in ss.split(X):
    train_df = df.iloc[train_idx]
    validation_df = df.iloc[validation_idx]

We will establish a baseline by keeping all features.

In [13]:
y_validation = validation_df['SalePrice'].values
x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
y_train = train_df['SalePrice'].values
x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
lr.fit(x_train, y_train)
y_pred = lr.predict(x_validation)
baseline = rmse(y_validation, y_pred)
baseline

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

0.10890203559597167

This rmse of 0.10890203559597167 is actually quite a bit better (relatively speaking) than the baseline of 0.12503266782864195 over the whole dataset and the rmse of 0.11751015503521701488 we obtained by merely dropping points from that dataset.  This is because the R code has dropped the outliers before preprocessing and so several engineered features have been refit to this dataset. 

## Features

We have a collection of Features, some of which were identified to be of potentially low-quality in predicting the response, others of which are known to be highly correlated with other Features.  We want to identify subsets of features that we can drop to improve the regression.

In [14]:
drop_cands = [
    'LotFrontage', 'LotArea', 'BsmtUnfSF', 'LowQualFinSF',
    'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AllSizesSumLin', 'AreasSum',
    'X1stFlrSF','X1stLin', 'X2ndFlrSF', 'X2ndLin',
    'TotalBath', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'Age', 'AgeLin', 'RemodAgeLin','RemodAge',
    'MasVnrArea', 'MasVnrAreaLin',
    'DeckPorchLin','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch'
]

In [17]:
corr_df = df[drop_cands].corr()

In [19]:
corr_df[corr_df > 0.75]

LotFrontage                 LotArea  BsmtUnfSF  LowQualFinSF  \
LotFrontage             1.0                     NaN        NaN           NaN   
LotArea                 NaN  1.00000000000000000000        NaN           NaN   
BsmtUnfSF               NaN                     NaN        1.0           NaN   
LowQualFinSF            NaN                     NaN        NaN           1.0   
LogGrLivArea            NaN                     NaN        NaN           NaN   
GrLivArea               NaN                     NaN        NaN           NaN   
TotalHouseArea          NaN                     NaN        NaN           NaN   
LivArea                 NaN                     NaN        NaN           NaN   
LivAreaWt               NaN                     NaN        NaN           NaN   
AllSizesSum             NaN  0.98574282164136073359        NaN           NaN   
AllSizesSumLin          NaN                     NaN        NaN           NaN   
AreasSum                NaN                     NaN        NaN           NaN   
X1stFlrSF               NaN                     NaN        NaN           NaN   
X1stLin                 NaN                     NaN        NaN           NaN   
X2ndFlrSF               NaN                     NaN        NaN           NaN   
X2ndLin                 NaN                     NaN        NaN           NaN   
TotalBath               NaN                     NaN        NaN           NaN   
BsmtFullBath            NaN                     NaN        NaN           NaN   
BsmtHalfBath            NaN                     NaN        NaN           NaN   
FullBath                NaN                     NaN        NaN           NaN   
HalfBath                NaN                     NaN        NaN           NaN   
Age                     NaN                     NaN        NaN           NaN   
AgeLin                  NaN                     NaN        NaN           NaN   
RemodAgeLin             NaN                     NaN        NaN           NaN   
RemodAge                NaN                     NaN        NaN           NaN   
MasVnrArea              NaN                     NaN        NaN           NaN   
MasVnrAreaLin           NaN                     NaN        NaN           NaN   
DeckPorchLin            NaN                     NaN        NaN           NaN   
WoodDeckSF              NaN                     NaN        NaN           NaN   
OpenPorchSF             NaN                     NaN        NaN           NaN   
EnclosedPorch           NaN                     NaN        NaN           NaN   
X3SsnPorch              NaN                     NaN        NaN           NaN   
ScreenPorch             NaN                     NaN        NaN           NaN   

                          LogGrLivArea               GrLivArea  \
LotFrontage                        NaN                     NaN   
LotArea                            NaN                     NaN   
BsmtUnfSF                          NaN                     NaN   
LowQualFinSF                       NaN                     NaN   
LogGrLivArea    1.00000000000000000000  0.97284711350461094970   
GrLivArea       0.97284711350461094970  1.00000000000000000000   
TotalHouseArea  0.86317256580400802957  0.83274164265266448215   
LivArea         0.76853554865027584597  0.76993518450779729889   
LivAreaWt       0.92084699562381455706  0.91418427872678065249   
AllSizesSum                        NaN                     NaN   
AllSizesSumLin                     NaN                     NaN   
AreasSum        0.83477182372955105460  0.82112328298677050853   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
X2ndFlrSF                          NaN                     NaN   
X2ndLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
BsmtFullBath                       NaN                     NaN   
BsmtHalfBath                       NaN   

We'll restrict our drop set to the highly correlated features to make this more readable,

In [29]:
drop_cands = [
    'LotArea', 'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AreasSum',
    'X1stFlrSF','X1stLin', 'TotalBath', 'FullBath', 'MasVnrArea', 'MasVnrAreaLin'
]

In [30]:
corr_df = df[drop_cands].corr()

In [31]:
corr_df[corr_df > 0.75]

LotArea            LogGrLivArea  \
LotArea         1.00000000000000000000                     NaN   
LogGrLivArea                       NaN  1.00000000000000000000   
GrLivArea                          NaN  0.97284711350461094970   
TotalHouseArea                     NaN  0.86317256580400802957   
LivArea                            NaN  0.76853554865027584597   
LivAreaWt                          NaN  0.92084699562381455706   
AllSizesSum     0.98574282164136073359                     NaN   
AreasSum                           NaN  0.83477182372955105460   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                             GrLivArea          TotalHouseArea  \
LotArea                            NaN                     NaN   
LogGrLivArea    0.97284711350461094970  0.86317256580400802957   
GrLivArea       1.00000000000000000000  0.83274164265266448215   
TotalHouseArea  0.83274164265266448215  1.00000000000000000000   
LivArea         0.76993518450779729889  0.80504876423853111156   
LivAreaWt       0.91418427872678065249  0.87721491992228128876   
AllSizesSum                        NaN                     NaN   
AreasSum        0.82112328298677050853  0.93168430704452442903   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                               LivArea               LivAreaWt  \
LotArea                            NaN                     NaN   
LogGrLivArea    0.76853554865027584597  0.92084699562381455706   
GrLivArea       0.76993518450779729889  0.91418427872678065249   
TotalHouseArea  0.80504876423853111156  0.87721491992228128876   
LivArea         1.00000000000000000000  0.94672752095422807450   
LivAreaWt       0.94672752095422807450  1.00000000000000000000   
AllSizesSum                        NaN                     NaN   
AreasSum        0.79843485489696608415  0.86392271466388026457   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                           AllSizesSum                AreasSum  \
LotArea         0.98574282164136073359                     NaN   
LogGrLivArea                       NaN  0.83477182372955105460   
GrLivArea                          NaN  0.82112328298677050853   
TotalHouseArea                     NaN  0.93168430704452442903   
LivArea                            NaN  0.79843485489696608415   
LivAreaWt                          NaN  0.86392271466388026457   
AllSizesSum     1.00000000000000000000                     NaN   
AreasSum                           NaN  1.00000000000000000000   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                             X1stFlrSF                 X1stLin  \
L

Let's first compare the pairs.

In [32]:
drop_cands = ['LotArea', 'AllSizesSum']

In [36]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

In [37]:
output_df

Dropped                    RMSE          Diff from Base
0                          ()  0.10890203559597166905  0.00000000000000000000
2            ('AllSizesSum',)  0.10907261155010689047  0.00017057595413522142
3  ('LotArea', 'AllSizesSum')  0.10923860873748110489  0.00033657314150943585
1                ('LotArea',)  0.10941263205625721688  0.00051059646028554784

We should keep these features.

In [38]:
drop_cands = ['X1stFlrSF','X1stLin']

In [39]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

In [40]:
output_df

Dropped                    RMSE          Diff from Base
0                        ()  0.10890203559597166905  0.00000000000000000000
1            ('X1stFlrSF',)  0.10915140987981494791  0.00024937428384327887
2              ('X1stLin',)  0.10915466658176065096  0.00025263098578898191
3  ('X1stFlrSF', 'X1stLin')  0.10931604721237808686  0.00041401161640641782

In [41]:
drop_cands = ['TotalBath', 'FullBath']

In [42]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

In [43]:
output_df

Dropped                    RMSE          Diff from Base
3  ('TotalBath', 'FullBath')  0.10881350768541774021 -0.00008852791055392883
0                         ()  0.10890203559597166905  0.00000000000000000000
1             ('TotalBath',)  0.10936817800946713186  0.00046614241349546282
2              ('FullBath',)  0.10941224559384052362  0.00051020999786885457

The gain by dropping both of these isn't big enough to consider dropping a predictor that we'd consider strong by hedonic reasoning.

In [44]:
drop_cands = ['MasVnrArea', 'MasVnrAreaLin']

In [45]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

In [46]:
output_df

Dropped                    RMSE  \
0                               ()  0.10890203559597166905   
3  ('MasVnrArea', 'MasVnrAreaLin')  0.10891541003492731521   
2               ('MasVnrAreaLin',)  0.10896847275158183521   
1                  ('MasVnrArea',)  0.10900871181861146453   

           Diff from Base  
0  0.00000000000000000000  
3  0.00001337443895564616  
2  0.00006643715561016617  
1  0.00010667622263979548

In [47]:
drop_cands = ['LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum']

In [48]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 115 iterations, i.e. alpha=1.048e-04, with an active set of 109 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 133 iterations, i.e. alpha=8.760e-05, with an active set of 119 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 163 iterations, i.e. alpha=5.373e-05, with an active set of 147 regressors, and the smallest cholesky pivot element being 3.799e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model

In [49]:
output_df

Dropped  \
50                          ('LogGrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')   
28                                       ('LogGrLivArea', 'TotalHouseArea', 'AreasSum')   
59             ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')   
44                          ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'AreasSum')   
58               ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
49                            ('LogGrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
39                                            ('TotalHouseArea', 'LivArea', 'AreasSum')   
31                                            ('LogGrLivArea', 'LivAreaWt', 'AreasSum')   
21                                                            ('LivAreaWt', 'AreasSum')   
47                               ('LogGrLivArea', 'GrLivArea', 'LivAreaWt', 'AreasSum')   
11                                                         ('LogGrLivArea', 'AreasSum')   
6                                                                         ('AreasSum',)   
61               ('LogGrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
30                                              ('LogGrLivArea', 'LivArea', 'AreasSum')   
51                                 ('LogGrLivArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
20                                                              ('LivArea', 'AreasSum')   
40                                          ('TotalHouseArea', 'LivAreaWt', 'AreasSum')   
36                                                 ('GrLivArea', 'LivArea', 'AreasSum')   
56                               ('TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
29                                             ('LogGrLivArea', 'LivArea', 'LivAreaWt')   
46                                 ('LogGrLivArea', 'GrLivArea', 'LivArea', 'AreasSum')   
15                                                            ('GrLivArea', 'AreasSum')   
34                                          ('GrLivArea', 'TotalHouseArea', 'AreasSum')   
63  ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
53                               ('GrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
60                    ('LogGrLivArea', 'GrLivArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
45                                ('LogGrLivArea', 'GrLivArea', 'LivArea', 'LivAreaWt')   
5                                                                        ('LivAreaWt',)   
24                                           ('LogGrLivArea', 'GrLivArea', 'LivAreaWt')   
25                                            ('LogGrLivArea', 'GrLivArea', 'AreasSum')   
..                                                                                  ...   
0                                                                                    ()   
8                                                    ('LogGrLivArea', 'TotalHouseArea')   
10                                                        ('LogGrLivArea', 'LivAreaWt')   
3                                                                   ('TotalHouseArea',)   
18                                                       ('TotalHouseArea', 'AreasSum')   
27                                      ('LogGrLivArea', 'TotalHouseArea', 'LivAreaWt')   
17                                                      ('TotalHouseArea', 'LivAreaWt')   
2                                                                        ('GrLivArea',)   
26                                        ('LogGrLivArea', 'TotalHouseArea', 'LivArea')   
16                                                        ('TotalHouseArea', 'LivArea')   
48                           ('LogGrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt')   
19                                                             ('LivArea', 'LivAreaWt')   
12                                                      ('GrLivArea', 'Total

In [50]:
output_df.to_csv('col_drop_results_111001001.csv', header=True)

The best result, dropping  	('LogGrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum') is a modest gain.  Again, we can choose other top candiates to generate additional datasets to use in an ensemble.